In [1]:
import sys
sys.getdefaultencoding()

'ascii'

In [2]:
#reload(sys)
#sys.setdefaultencoding('utf-8')

In [3]:
from sklearn.externals import joblib

In [4]:

clf = joblib.load('classify.model')
cate_dict = joblib.load('cate_dict.dat')
vectorizer = joblib.load('vectorizer.dat')


In [5]:
joblib.dump(clf,'n_classify.model')

['n_classify.model',
 'n_classify.model_01.npy',
 'n_classify.model_02.npy',
 'n_classify.model_03.npy']

In [6]:
joblib.dump(cate_dict,'n_cate_dict.dat')
joblib.dump(vectorizer,'n_vectorizer.dat')

['n_vectorizer.dat']

In [7]:
cate_id_name_dict = dict(map(lambda kv:(kv[1],kv[0]),cate_dict.items()))

In [8]:
pred = clf.predict(vectorizer.transform(['[신한카드5%할인][서우한복] 아동한복 여자아동 금나래 (분홍)']))[0]
print(cate_id_name_dict[pred])

패션의류;아동의류;한복


In [9]:
image_all_cache = joblib.load('image_cache/image_all_dict.cache')

In [10]:
image_all_cache

{'8284053019.jpg': [('bolo tie', '0.52997'),
  ('chain', '0.13735'),
  ('stethoscope', '0.11027'),
  ('loupe', '0.06622'),
  ('necklace', '0.04344')],
 '8260116297.jpg': [('espresso maker', '0.17685'),
  ('bucket', '0.15010'),
  ('mixing bowl', '0.06116'),
  ('cocktail shaker', '0.05880'),
  ('water jug', '0.04871')],
 '7704257593.jpg': [('maillot', '0.27543'),
  ('maillot', '0.24505'),
  ('brassiere', '0.08476'),
  ('dumbbell', '0.07860'),
  ('barbell', '0.07202')],
 '8352832425.jpg': [('chain mail', '0.55674'),
  ('stole', '0.09517'),
  ('cardigan', '0.05063'),
  ('miniskirt', '0.04581'),
  ('wool', '0.02792')],
 '8082224988.jpg': [('loudspeaker', '0.16636'),
  ('radio', '0.11706'),
  ('loupe', '0.10191'),
  ('reflex camera', '0.09304'),
  ('projector', '0.07434')],
 '8240711783.20150123095423.jpg': [('lighter', '0.08118'),
  ('scoreboard', '0.05932'),
  ('modem', '0.05834'),
  ('hard disc', '0.04287'),
  ('radio', '0.03725')],
 '8270242743.jpg': [('switch', '0.10371'),
  ('analog cl

In [11]:
d_all = joblib.load('image_cache/d_all.cache')

In [12]:
d_all

{u'[\ube44\ube44\uc548]\uac00\ubcbc\uc6b4\uc18c\uc7ac\uac70\ub4e4\ud32c\ud2f0(GP3186)': u'7613045745.jpg',
 u'[\ud328\uc158\ud50c\ub7ec\uc2a4][PANDORA by vivien]\ud310\ub3c4\ub77cby\ube44\ube44\uc548 \uc2a4\ud2b8\ub77c\uc774\ud504 \ud328\ud134 \uc0b0\uc804\uc6a9 \ube0c\ub77c\ud32c\ud2f0\uc138\ud2b8 bp1547_a': u'7603000429.jpg',
 u'[\uac8c\uc2a4\ud0a4\uc988]\ub808\uc774\uc2a4 \uc7a5\uc2dd \uc18d\ubc14\uc9c0 G54KCP040': u'8433726625.jpg',
 u'\ud1a0\ud0a4\uc624 \ub9ac\ud30c\uc778\ub4dc \uc2a4\ud2b8\ub77c\uc774\ud504 \uc2ac\ub799\uc2a4 (\ube14\ub799)': u'8337996599.jpg',
 u'[4%\uc989\uc2dc\ud560\uc778\ucfe0\ud3f0](\uc815\ud488)\uce90\ub17c 100D   18-55 IS ll   50.8   55-250mmIS': u'8277349444.jpg',
 u'[\ud574\uc678]Panasonic hand mixer MK-H4-W white': u'8260100294.jpg',
 u'[\ud544\ub9bd\uc2a4 \uc815\ud488]HX-6013/\uc804\ub3d9\uce6b\uc194\ubaa8/\uce6b\uc194\ubaa8/\ud544\ub9bd\uc2a4\uc804\ub3d9\uce6b\uc194\ubaa8/\ud544\ub9bd\uc2a4\uce6b\uc194\ubaa8/\uc18c\ub2c8\ucf00\uc5b4\uce6b\uc194\ubaa8/

In [13]:
d_cache = {}

In [ ]:
from bottle import route, run, template,request,get, post
from konlpy.tag import Twitter
import  time
from threading import  Condition

_CONDITION = Condition()
twitter = Twitter()
@route('/classify')
def classify():
    print("classify called")
    img = request.GET.get('img','')
    name = request.GET.get('name', '').decode('utf-8')
    if name in d_all:
        if d_all[name] in image_all_cache:
            name = name + image_all_cache[d_all[name]][0][0]
    if name in d_cache:
        result = d_cache[name]
    else:
        parsed = twitter.pos(name)
        r_list = []
        for each in parsed:
#            if each[1] == 'Noun' or each[1] == 'Alpha':
            r_list.append(each[0])

        result = ' '.join(r_list)
        d_cache[name] = result
        joblib.dump(d_cache,'d_cache.cache')
    pred = clf.predict(vectorizer.transform([result]))[0]
    return {'cate':cate_id_name_dict[pred]}

run(host='0.0.0.0', port=8887)


Bottle v0.12.9 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8887/
Hit Ctrl-C to quit.



classify called


121.140.65.88 - - [22/Aug/2016 08:11:27] "GET /classify?name=%EC%9D%B8%ED%85%94CPU%EC%BD%94%EC%96%B4%20i3-4%EC%84%B8%EB%8C%80%20%ED%95%98%EC%8A%A4%EC%9B%B0%20%EB%A6%AC%ED%94%84%EB%A0%88%EC%8B%9C%204160(3.6GHz/INTEL)&img=http://shopping.phinf.naver.net/main_7975500/7975500771.jpg HTTP/1.1" 200 62


classify called


121.140.65.88 - - [22/Aug/2016 08:11:27] "GET /classify?name=[4%%EC%A6%89%EC%8B%9C%ED%95%A0%EC%9D%B8%EC%BF%A0%ED%8F%B0]%EC%9D%B8%ED%85%94%20intel%20%EC%BD%94%EC%96%B44%EC%84%B8%EB%8C%80%20%ED%95%98%EC%8A%A4%EC%9B%B0%20i3-4160&img=http://shopping.phinf.naver.net/main_8350292/8350292853.jpg HTTP/1.1" 200 62


...

121.140.65.88 - - [22/Aug/2016 08:13:05] "GET /classify?name=[%ED%95%B4%EC%99%B8][%EA%B5%AC%EB%A7%A4%EB%8C%80%ED%96%89][B00AZVG9NQ]%20AD5307BRUZ&img=http://shopping.phinf.naver.net/main_8266271/8266271076.jpg HTTP/1.1" 200 72
121.140.65.88 - - [22/Aug/2016 08:13:05] "GET /classify?name=[%ED%95%B4%EC%99%B8]Western%20DigitalWestern%20Digital%20WD%20TV%20Live%20Media%20Player%20Wi-fi%201080p&img=http://shopping.phinf.naver.net/main_8275627/8275627277.jpg HTTP/1.1" 200 115


classify called
classify called


121.140.65.88 - - [22/Aug/2016 08:13:06] "GET /classify?name=[%ED%95%B4%EC%99%B8][%EA%B5%AC%EB%A7%A4%EB%8C%80%ED%96%89][B00MMV4FMY]%20Voltage%20References%20Ultracompact%20Prec%203V%20(5%20pieces)&img=http://shopping.phinf.naver.net/main_8266191/8266191335.jpg HTTP/1.1" 200 72
121.140.65.88 - - [22/Aug/2016 08:13:06] "GET /classify?name=Opticis%20HD%EB%AF%B8%EB%94%94%EC%96%B4%20%ED%94%8C%EB%A0%88%EC%9D%B4%EC%96%B4(%EB%84%A4%ED%8A%B8%EC%9B%8C%ED%81%AC%EC%A7%80%EC%9B%90)HDMP-1000&img=http://shopping.phinf.naver.net/main_8235341/8235341743.jpg HTTP/1.1" 200 115


classify called
classify called


121.140.65.88 - - [22/Aug/2016 08:13:06] "GET /classify?name=[%EC%A0%95%ED%92%88]%20%EB%8B%88%EC%BD%98%20DSLR%20D800E%20Body+%EB%A7%A8%ED%94%84%EB%A1%9C%ED%86%A0%20MKBFRA4-BH%20%EC%82%BC%EA%B0%81%EB%8C%80%20+%EC%9D%B8%ED%99%94%EA%B6%8C+%EC%9B%B9%ED%95%98%EB%93%9C&img=http://shopping.phinf.naver.net/main_8303716/8303716371.jpg HTTP/1.1" 200 116
121.140.65.88 - - [22/Aug/2016 08:13:06] "GET /classify?name=[NETGEAR%20PTV3000]%20Intel%20WiDi%20%ED%83%91%EC%9E%AC%EB%90%9C%20%EB%85%B8%ED%8A%B8%EB%B6%81%EC%97%90%EC%84%9C%20%EB%8C%80%ED%98%95%EC%8A%A4%ED%81%AC%EB%A6%B0TV%20%ED%92%80HD%ED%99%94%EB%A9%B4%EC%9C%BC%EB%A1%9C%20%EB%AF%B8%EB%9D%BC%EC%BA%90%EC%8A%A4%ED%8A%B8%20%EC%A7%80%EC%9B%90&img=http://shopping.phinf.naver.net/main_8242176/8242176723.jpg HTTP/1.1" 200 101


 * 추후 여기 docker 에서 뭔가 python package 설치할게 있으면 
 * /opt/conda/bin/pip2 install bottle 이런식으로 설치 가능

In [ ]:
d_list